# Machine learning on wine

**Topics:** Text analysis, linear regression, logistic regression, text analysis, classification

**Datasets**

- **wine-reviews.csv** Wine reviews scraped from https://www.winemag.com/
- **Data dictionary:** just go [here](https://www.winemag.com/buying-guide/tenuta-dellornellaia-2007-masseto-merlot-toscana/) and look at the page

## The background

You work in the **worst newsroom in the world**, and you've had a hard few weeks at work - a couple stories killed, a few scoops stolen out from under you. It's not going well.

And because things just can't get any worse: your boss shows up, carrying a huge binder. She slams it down on your desk.

"You know some machine learning stuff, right?"

You say "no," but she isn't listening. She's giving you an assignment, the _worst assignment_:

> Machine learning is the new maps. Let's get some hits!
>
> **Do some machine learning on this stuff.**

"This stuff" is wine reviews.

## A tiny, meagre bit of help

You have a dataset. It has some stuff in it:

* **Numbers:**
    - Year published
    - Alcohol percentage
    - Price
    - Score
    - Bottle size
* **Categories:**
    - Red vs white
    - Different countries
    - Importer
    - Designation
    - Taster
    - Variety
    - Winery
* **Free text:**
    - Wine description

# Cleaning up your data

Many of these pieces - the alcohol, the year produced, the bottle size, the country the wine is from - aren't in a format you can use. Convert the ones to numbers that are numbers, and extract the others from the appropriate strings.

In [1]:
import pandas as pd
import fuzzy_pandas as fpd

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
import numpy as np

In [4]:
df = pd.read_csv("wine-reviews.csv")
df.head()

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5%,750 ml,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review]
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
2,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerro...,This is another fine vintage for this rare win...,Paul Gregutt,"$25, Buy Now",Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
3,https://www.winemag.com/buying-guide/jcb-2011-...,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),Light in color and lilting floral aromas of ro...,Virginie Boone,"$65, Buy Now",No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,13%,750 ml,Red,NaN,12/1/2014,Not rated yet [Add Your Review]
4,https://www.winemag.com/buying-guide/pazo-pond...,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, me...",Michael Schachner,"$17, Buy Now",NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,13%,750 ml,White,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review]


In [5]:
df['bottle size'].value_counts()

750 ml    18342
750ML      2713
375 ml      209
500 ml       86
375ML        28
3 L          15
1.5 L        15
1 L          13
500ML         8
187 ml        2
3L            2
1L            1
1.5L          1
Name: bottle size, dtype: int64

In [6]:
r = {'750 ml':750, '750ML':750, 
    '375 ml':375, '500 ml':500, '500ML':500,
     '1 L':1000, '1.5 L':1500,
       '375ML':375, '3 L':3000,
     '1.5L':1500, '1L':1000,
     '3L':3000, '187 ml':187}

df['bottle_size_ml'] = df['bottle size'].replace(r)
df.bottle_size_ml.value_counts()

750     21055
375       237
500        94
3000       17
1500       16
1000       14
187         2
Name: bottle_size_ml, dtype: int64

In [7]:
df['price_clean'] = df.price.str.split(',', expand=True)[0].str.replace('$', '').replace('N/A', np.nan).astype(float)

In [8]:
df['avg_rating_clean'] = df['user avg rating'].str[:2].replace('No', np.nan)

In [9]:
df['alcohol_clean'] = df.alcohol.str.replace('%', '').astype(float)

In [10]:
df['year'] = df.wine_name.str.extract(r'.*?(\d\d\d\d)').astype(float)

In [12]:
df['country'] = df.appellation.str.extract(r'(\w.*)$').astype(str)

In [ ]:
df.loc[df['year']<1900,'year']=df[df['year']<1900].wine_name.str.extract(r'.*?\d\d\d\d.*?(\d\d\d\d)',expand=False).astype(float)

In [13]:
df.variety.value_counts()

Pinot Noir                                     2429
Chardonnay                                     1991
Cabernet Sauvignon                             1847
Red Blends, Red Blends                         1226
Bordeaux-style Red Blend                       1035
Syrah                                           964
Riesling                                        849
Sauvignon Blanc                                 759
Merlot                                          643
Rosé                                            536
Zinfandel                                       498
Malbec                                          429
Champagne Blend, Sparkling                      386
Portuguese Red                                  363
Tempranillo                                     357
White Blend                                     342
Sparkling Blend, Sparkling                      338
Nebbiolo                                        300
Grüner Veltliner                                287
Rhône-style 

## What might be interesting in this dataset?

Maybe start out playing around _without_ machine learning. Here are some thoughts to get you started:

* I've heard that since the 90's wine has gone through [Parkerization](https://www.estatewinebrokers.com/blog/the-parkerization-of-wine-in-the-1990s-and-beyond/), an increase in production of high-alcohol, fruity red wines thanks to the influence of wine critic Robert Parker.
* Red and white wines taste different, obviously, but people always use [goofy words to describe them](https://winefolly.com/tutorial/40-wine-descriptions/)
* Once upon a time in 1976 [California wines proved themselves against France](https://en.wikipedia.org/wiki/Judgment_of_Paris_(wine)) and France got very angry about it

In [14]:
df.wine_points.value_counts()

90.0     2312
87.0     2280
92.0     1954
88.0     1839
93.0     1767
91.0     1746
86.0     1663
84.0     1436
94.0     1263
89.0     1139
85.0      943
83.0      929
82.0      680
95.0      635
81.0      257
96.0      231
80.0      151
97.0      136
98.0       46
99.0       17
100.0      11
Name: wine_points, dtype: int64

## But machine learning?

Well, you can usually break machine learning down into a few different things. These aren't necessarily perfect ways of categorizing things, but eh, close enough.

* **Predicting a number**
    - Linear regression
    - How does a change in unemployment translate into a change in life expectancy?
* **Predicting a category** (aka classification)
    - Lots of algos options: logistic regression, random forest, etc
    - For example, predicting cuisines based on ingredients
* **Seeing what influences a numeric outcome**
    - Linear regression since the output is a number
    - For example, minority and poverty status on test scores 
* **Seeing what influences a categorical outcome**
    - Logistic regression since the output is a category
    - Race and car speed for if you get a waring vs ticket
    - Wet/dry pavement and car weight if you survive or not in a car crash)

We have numbers, we have categories, we have all sorts of stuff. **What are some ways we can mash them together and use machine learning?**

### Brainstorm some ideas

Use the categories above to try to come up with some ideas. Be sure to scroll up where I break down categories vs numbers vs text!

**I'll give you one idea for free:** if you don't have any ideas, start off by creating a classifier that determines whether a wine is white or red based on the wine's description.

You can also go to https://library.columbia.edu and see if you can find some academic papers about wine. I'm sure they'll inspire you! (and they might even have some ML ideas in them you can steal, too)

# Implement 2 of your machine learning ideas

In [15]:
df['grapes'] = df.variety.str.replace('(,.*)$','').astype(str)

In [17]:
df = df[~df['grapes'].str.contains('Blend')]

In [18]:
df = df[~df['grapes'].str.contains('Portuguese')]

In [20]:
df.head()

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,...,importer,date published,user avg rating,bottle_size_ml,price_clean,avg_rating_clean,alcohol_clean,year,country,grapes
0,https://www.winemag.com/buying-guide/artadi-20...,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black pl...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,...,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review],750,25.0,NaN,14.5,2011.0,"Rioja, Northern Spain, Spain",Tempranillo
1,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Du...,"A tiny production wine, this is rich, tart and...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,...,NaN,12/1/2014,Not rated yet [Add Your Review],750,65.0,NaN,13.5,2012.0,"Dundee Hills, Willamette Valley, Oregon, US",Chardonnay
2,https://www.winemag.com/buying-guide/adelsheim...,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerro...,This is another fine vintage for this rare win...,Paul Gregutt,"$25, Buy Now",Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,...,NaN,12/1/2014,Not rated yet [Add Your Review],750,25.0,NaN,13.5,2013.0,"Ribbon Ridge, Willamette Valley, Oregon, US",Auxerrois
3,https://www.winemag.com/buying-guide/jcb-2011-...,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),Light in color and lilting floral aromas of ro...,Virginie Boone,"$65, Buy Now",No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,...,NaN,12/1/2014,Not rated yet [Add Your Review],750,65.0,NaN,13.0,2011.0,"Sonoma Coast, Sonoma, California, US",Pinot Noir
4,https://www.winemag.com/buying-guide/pazo-pond...,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, me...",Michael Schachner,"$17, Buy Now",NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,...,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review],750,17.0,NaN,13.0,2013.0,"Rías Baixas, Galicia, Spain",Albariño


In [21]:
df.grapes.value_counts()

Pinot Noir                         2429
Chardonnay                         1991
Cabernet Sauvignon                 1847
Syrah                               964
Riesling                            849
Sauvignon Blanc                     759
Merlot                              643
Rosé                                536
Zinfandel                           498
Malbec                              429
Tempranillo                         357
Nebbiolo                            300
Grüner Veltliner                    287
Shiraz                              235
Sangiovese                          229
Pinot Gris                          197
Viognier                            191
Cabernet Franc                      182
Chenin Blanc                        163
Gewürztraminer                      154
Barbera                             153
Pinot Grigio                        143
Petite Sirah                        137
Port                                134
Gamay                               123


In [22]:
df.sort_values(by='grapes')

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,...,importer,date published,user avg rating,bottle_size_ml,price_clean,avg_rating_clean,alcohol_clean,year,country,grapes
929,https://www.winemag.com/buying-guide/zacharias...,85.0,Zacharias 2012 Agiorgitiko (Nemea),"Plum, spice and red berries on the nose and pa...",Susan Kostrzewa,"$13, Buy Now",NaN,"Agiorgitiko, Greek Red","Nemea, Greece",Zacharias,...,"Stellar Importing Company, LLC",8/1/2015,Not rated yet [Add Your Review],750,13.0,NaN,12.5,2012.0,"Nemea, Greece",Agiorgitiko
15266,https://www.winemag.com/buying-guide/kouros-20...,80.0,Kouros 2000 Agiorgitiko (Corinth),NaN,Joe Czerwinski,"$9, Buy Now",NaN,"Agiorgitiko, Greek Red","Corinth, Greece",Kouros,...,Nestor Imports,9/1/2004,Not rated yet [Add Your Review],750,9.0,NaN,12.0,2000.0,"Corinth, Greece",Agiorgitiko
15846,https://www.winemag.com/buying-guide/ktima-dri...,85.0,Ktima Driopi 2005 Agiorgitiko (Nemea),"This earthy red starts with aromas of leather,...",Susan Kostrzewa,"$25, Buy Now",NaN,"Agiorgitiko, Greek Red","Nemea, Greece",Ktima Driopi,...,Angel's Share Wines,4/1/2010,Not rated yet [Add Your Review],750,25.0,NaN,13.0,2005.0,"Nemea, Greece",Agiorgitiko
15267,https://www.winemag.com/buying-guide/spiropoul...,80.0,Spiropoulos 2000 Red Stag Agiorgitiko (Pelopon...,NaN,Joe Czerwinski,"$15, Buy Now",Red Stag,"Agiorgitiko, Greek Red","Peloponnese, Greece",Spiropoulos,...,Athenee Importers,9/1/2004,Not rated yet [Add Your Review],750,15.0,NaN,12.5,2000.0,"Peloponnese, Greece",Agiorgitiko
13074,https://www.winemag.com/buying-guide/achaia-cl...,83.0,Achaia Clauss 2000 Agiorgitiko (Corinth),NaN,Joe Czerwinski,"$10, Buy Now",NaN,"Agiorgitiko, Greek Red","Corinth, Greece",Achaia Clauss,...,"Stellar Importing Company, LLC",9/1/2004,Not rated yet [Add Your Review],750,10.0,NaN,12.0,2000.0,"Corinth, Greece",Agiorgitiko
8963,https://www.winemag.com/buying-guide/alexandro...,86.0,Alexandros Megapanos 2005 Megapanos Agiorgitik...,Sweet cedar and cinnamon aromas and flavors of...,Susan Kostrzewa,"$28, Buy Now",Megapanos,"Agiorgitiko, Greek Red","Nemea, Greece",Alexandros Megapanos,...,Wonderful Ethnic Imports,4/1/2010,Not rated yet [Add Your Review],750,28.0,NaN,12.5,2005.0,"Nemea, Greece",Agiorgitiko
10414,https://www.winemag.com/buying-guide/spiropoul...,86.0,Spiropoulos 2006 Red Stag Agiorgitiko (Nemea),"Ripe cherry, cedar, vanilla and spice aromas a...",Susan Kostrzewa,"$15, Buy Now",Red Stag,"Agiorgitiko, Greek Red","Nemea, Greece",Spiropoulos,...,Athenee Importers,4/1/2010,Not rated yet [Add Your Review],750,15.0,NaN,13.0,2006.0,"Nemea, Greece",Agiorgitiko
3946,https://www.winemag.com/buying-guide/tsantali-...,87.0,Tsantali 2003 Réserve Agiorgitiko (Nemea),"Agiorgitiko, famed in Nemea and popular for it...",Susan Kostrzewa,"$16, Buy Now",Réserve,"Agiorgitiko, Greek Red","Nemea, Greece",Tsantali,...,"Fantis Imports, Inc",8/1/2008,Not rated yet [Add Your Review],750,16.0,NaN,13.5,2003.0,"Nemea, Greece",Agiorgitiko
7719,https://www.winemag.com/buying-guide/estate-bi...,88.0,Estate Biblia Chora 2008 Areti Agiorgitiko (Pa...,This expressive red starts with aromas of rasp...,Susan Kostrzewa,"$29, Buy Now",Areti,"Agiorgitiko, Greek Red","Pangeon, Greece",Estate Biblia Chora,...,Cava Spiliadis,9/1/2012,Not rated yet [Add Your Review],750,29.0,NaN,13.5,2008.0,"Pangeon, Greece",Agiorgitiko
6988,https://www.winemag.com/buying-guide/estate-co...,83.0,Estate Constantin Gofas 2005 Mythic River Agio...,This rustic red starts with a nose of red berr...,Susan Kostrzewa,"$13, Buy Now",Mythic River,"Agiorgitiko, Greek Red","Nemea, Greece",Estate Constantin Gofas,...,Athena Importing Co,8/1/2009,Not rated yet [Add Your Review],750,13.0,NaN,13.5,2005.0,"Nemea, Greece",Agiorgitiko
